In [ ]:
#pip install aiohttp
#1 time run

In [1]:
import csv
import requests
import time
import os
import json
import asyncio
import aiohttp
from aiohttp import ClientSession
import datetime

# All possible ranks defined by tiers and divisions
# "pages" is the first 2 pages of ranked players in each tier and division


In [ ]:
tiers=["IRON","BRONZE","SILVER","GOLD","PLATINUM","EMERALD","DIAMOND"]
divisions = ["I", "II", "III", "IV"]
pages=["1","2"]

fetch_and_save_ranked_players loops through all possible combinations of tiers combined with divisions and fetches the first 2 pages. It is just called using the previously instantiated arrays.

It should only be used if a big sample of players is needed.
The api_key is currently the personal API key from LeaguesDeveloper and will most likely need to be updated.

In the base_url also contains a region parameter, currently kr for korea. Changed it to match desired region

Finally, file_path defines the desire location for saving and storing all of the summonernames of the players in a csv file. It also saved tier and division.

In [5]:
def fetch_and_save_ranked_players(tiers, divisions, pages):
    api_key = "RGAPI-8840c4e8-ef3d-40cb-90fa-2ea005f2bb1c"  # Use your actual API key
    base_url = "https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={page}"
    headers = {
        "X-Riot-Token": api_key
    }
    
    file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'
    
    with open(file_path, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['tier', 'division', 'summonerName'])  # CSV Header

        for tier in tiers:
            for division in divisions:
                for page in pages:
                    url = base_url.format(tier=tier, division=division, page=page)
                    response = requests.get(url, headers=headers)

                    # Wait for 2 seconds before making the next request
                    time.sleep(2)

                    if response.status_code == 200:
                        players = response.json()
                        for player in players:
                            if player['wins'] + player['losses'] > 20:
                                writer.writerow([player['tier'], player['rank'], player['summonerName']])
                    else:
                        print(f"Error: {response.status_code}, {response.text}")


Example usage

In [6]:
#ONLY RUN IF NEW DATA REQUIRED
#made with personal apikey, therefore following personal pullrequest limitation
#fetch_and_save_ranked_players(tiers, divisions, pages)

find_duplicate_summoner_names_with_rows finds the dublicate summonernames from previous function, not editing anything. Should use same file_path 

In [9]:
#function for finding dublicates

def find_duplicate_summoner_names_with_rows(file_path):
    summoner_occurrences = {}  # Tracks summoner names and their row numbers
    duplicates_info = []  # Stores information about duplicates

    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row_number, row in enumerate(reader, start=1):  # Start counting rows from 1
            summoner_name = row['summonerName']
            if summoner_name in summoner_occurrences:
                # If the summoner name is already encountered, add the current row as a duplicate
                summoner_occurrences[summoner_name].append(row_number)
            else:
                # Otherwise, initialize the list with the current row number
                summoner_occurrences[summoner_name] = [row_number]

    # Filter out summoner names with more than one occurrence and prepare duplicate info
    for summoner_name, row_numbers in summoner_occurrences.items():
        if len(row_numbers) > 1:
            duplicates_info.append((summoner_name, row_numbers))

    return duplicates_info

# Specify the file path
file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'
duplicates = find_duplicate_summoner_names_with_rows(file_path)

# Example output
for summoner_name, rows in duplicates:
    print(f"Duplicate: {summoner_name} found in rows: {rows}")


Duplicate: 미라클제이티 found in rows: [67, 70]
Duplicate: KoreaJy found in rows: [68, 71]
Duplicate: 비챤 팬인 고라니 found in rows: [69, 72]
Duplicate: 무적대봉 found in rows: [225, 228]
Duplicate: Mute All Live found in rows: [226, 229]
Duplicate: 아개졸 found in rows: [227, 230]
Duplicate: 핸젤과그랬디야 found in rows: [381, 384]
Duplicate: 100363476del found in rows: [382, 385]
Duplicate: 오리새뀌 found in rows: [383, 386]
Duplicate: 삐뚤어진 또치 found in rows: [632, 634]
Duplicate: 껍질벗기기기장인 found in rows: [633, 635]
Duplicate: 기운찬곰텡 found in rows: [869, 874]
Duplicate: 미드보짓군영 found in rows: [870, 875]
Duplicate: Future maker KR found in rows: [871, 876]
Duplicate: 응급실카사노바 found in rows: [872, 877]
Duplicate: PAKACHU6 found in rows: [873, 878]
Duplicate: 백만송이라벤더 found in rows: [1071, 1073]
Duplicate: 꾸그밍 found in rows: [1072, 1074]
Duplicate: 간티드 found in rows: [1267, 1268]
Duplicate: XX황족미드XX found in rows: [1469, 1471]
Duplicate: 피키포킹 found in rows: [1470, 1472]
Duplicate: 우주의치킨 found in rows: [1679, 1683]
Duplica

remove_duplicate_summoner_names removes the duplicate rows.

In [10]:
#function for removing dublicates

def remove_duplicate_summoner_names(file_path):
    processed_summoners = set()  # To track unique summonerNames
    unique_rows = []  # To store rows after removing duplicates

    # Read the file and filter out duplicate summonerNames
    with open(file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            if row['summonerName'] not in processed_summoners:
                processed_summoners.add(row['summonerName'])
                unique_rows.append(row)

    # Write the unique rows back to the file
    with open(file_path, mode='w', newline='', encoding='utf-8') as outfile:
        fieldnames = ['tier', 'division', 'summonerName']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(unique_rows)

# Specify the file path
#file_path = '../DataProcessing/TestData/rankedPlayers.csv'
#remove_duplicate_summoner_names(file_path)


The following functions "split_csv_by_xxx_tier", separates all the ranked tiers into tier specific csv files.
The "source_file_path" the functions takes as a parameter is the same file from earlier. It should be the cleaned up version of the summonernames.
"file_name" and "file_path" is the desired name and location of the new file created, only containing the tier specific function.
Each of the following 7 functions can be called seperately.

In [ ]:
def split_csv_by_iron_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the IRON tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is IRON
            if tier == "IRON":
                file_name = "IRON_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                # alternative file_path='../DataProcessing/TestData/IRON_players_korea.csv
                
                # Check if we already have a writer for the IRON tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the IRON file
                writer.writerow(row)
    
    # Close the file for the IRON tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_iron_tier(source_file_path)

In [ ]:
def split_csv_by_bronze_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the BRONZE tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is BRONZE
            if tier == "BRONZE":
                file_name = "BRONZE_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the BRONZE tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the BRONZE file
                writer.writerow(row)
    
    # Close the file for the BRONZE tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_bronze_tier(source_file_path)

In [ ]:
def split_csv_by_silver_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the SILVER tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is SILVER
            if tier == "SILVER":
                file_name = "SILVER_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the SILVER tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the SILVER file
                writer.writerow(row)
    
    # Close the file for the SILVER tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_silver_tier(source_file_path)

In [ ]:
def split_csv_by_gold_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the GOLD tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is GOLD
            if tier == "GOLD":
                file_name = "GOLD_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the GOLD tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the GOLD file
                writer.writerow(row)
    
    # Close the file for the GOLD tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_gold_tier(source_file_path)

In [ ]:
def split_csv_by_platinum_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the PLATINUM tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is PLATINUM
            if tier == "PLATINUM":
                file_name = "PLATINUM_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the PLATINUM tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the PLATINUM file
                writer.writerow(row)
    
    # Close the file for the PLATINUM tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_platinum_tier(source_file_path)

In [ ]:
def split_csv_by_emerald_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the EMERALD tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is EMERALD
            if tier == "EMERALD":
                file_name = "EMERALD_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the EMERALD tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the EMERALD file
                writer.writerow(row)
    
    # Close the file for the EMERALD tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_emerald_tier(source_file_path)

In [21]:
def split_csv_by_diamond_tier(source_file_path):
    # Define the base folder for the new files
    base_folder = os.path.dirname(source_file_path)
    
    # Initialize a dictionary to keep track of the CSV writer for the DIAMOND tier
    writer = None
    file = None
    
    with open(source_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        for row in reader:
            tier = row['tier']
            
            # Proceed only if the tier is DIAMOND
            if tier == "DIAMOND":
                file_name = "DIAMOND_players_korea.csv"
                file_path = os.path.join(base_folder, file_name)
                
                # Check if we already have a writer for the DIAMOND tier, if not, create it
                if writer is None:
                    file = open(file_path, mode='w', newline='', encoding='utf-8')
                    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)
                    writer.writeheader()
                
                # Write the row to the DIAMOND file
                writer.writerow(row)
    
    # Close the file for the DIAMOND tier if it's open
    if file is not None:
        file.close()

# Specify the source file path
source_file_path = '../DataProcessing/TestData/rankedPlayersKorea.csv'

# Call the function
split_csv_by_diamond_tier(source_file_path)

get_summoner_names_from_csv loads summonernames into an array for futher processing. file_path is the path to one of the files previously created with "split_csv_by_xxx_tier"
The code cell below storages the names in the variable "summoner_names" which will be used later.

In [ ]:
# saves summonernames for further processiing
def get_summoner_names_from_csv(file_path):
    summoner_names = []
    
    with open(file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            summoner_names.append(row['summonerName'])
    
    return summoner_names

# Specify the path to your CSV file
file_path = '../DataProcessing/TestData/DIAMOND_players_korea.csv'
summoner_names = get_summoner_names_from_csv(file_path)
print(summoner_names)


"update_summoner_names_with_puuid" replaces the summonernames with puuid, which can be used for finding ranked games later

replace api_key with your apikey, and be aware of the url. It should be changed to desired region. The relevant ones is kr and eu

summoner_names is being used here, with the same filepath used to generate summoner_names

In [ ]:
#personal apikey function, replacing summonernames with puuid
#FINDS PUUID FROM SUMMONER NAME
def update_summoner_names_with_puuid(file_path, summoner_names):
    api_key = "your key"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    summoner_to_puuid = {}

    start_time = time.time()
    last_update_time = start_time
    total_names = len(summoner_names)
    names_processed = 0

    # Fetch puuid for each summoner name
    for name in summoner_names:
        url = f"https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}"
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            summoner_to_puuid[name] = data['puuid']
        else:
            print(f"Error fetching data for {name}: {response.status_code}")
        names_processed += 1
        current_time = time.time()

        # Check if 1 minute has passed for progress update
        if current_time - last_update_time >= 60:
            print(f"Progress: {names_processed}/{total_names} summoner names processed.")
            last_update_time = current_time

        time.sleep(1.5)  # Delay to comply with rate limit
    
    # Update progress after finishing all requests
    print(f"Progress: {names_processed}/{total_names} summoner names processed. Update complete.")

    # Now, replace summoner names with puuid in the CSV
    temp_file_path = file_path + ".tmp"
    with open(file_path, mode='r', encoding='utf-8') as infile, open(temp_file_path, mode='w', newline='', encoding='utf-8') as outfile:
        reader = csv.DictReader(infile)
        writer = csv.DictWriter(outfile, fieldnames=reader.fieldnames)
        writer.writeheader()
        for row in reader:
            if row['summonerName'] in summoner_to_puuid:
                row['summonerName'] = summoner_to_puuid[row['summonerName']]
            writer.writerow(row)
    
    # Replace original file with the updated temp file
    os.replace(temp_file_path, file_path)


# Call the function
update_summoner_names_with_puuid(file_path, summoner_names)



"fetch_matches_and_save" fetches the last 20 matches if possible from each of the puuids.

edit url from asia to europe and vice versa if necessary

parameters should be the previous file path for the file containing all the puuids and the path for desired csv file that should contain the matchids

In [33]:
def fetch_matches_and_save(source_csv_path, target_csv_path):
    api_key = "your apikey"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": api_key
    }
    
    start_time = time.time()
    last_update_time = start_time
    summoners_processed = 0

    # Ensure the target file directory exists
    os.makedirs(os.path.dirname(target_csv_path), exist_ok=True)

    with open(source_csv_path, mode='r', encoding='utf-8') as infile, open(target_csv_path, mode='w', newline='', encoding='utf-8') as outfile:
        csv_reader = csv.DictReader(infile)
        csv_writer = csv.writer(outfile)
        csv_writer.writerow(['MatchID'])  # Header for target CSV
        
        for row in csv_reader:
            puuid = row['summonerName']  # Assuming this column actually contains puuids
            url = f"https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=20"
            response = requests.get(url, headers=headers)
            
            if response.status_code == 200:
                match_ids = response.json()
                for match_id in match_ids:
                    csv_writer.writerow([match_id])
            else:
                print(f"Error fetching matches for {puuid}: {response.status_code}")
            
            summoners_processed += 1
            current_time = time.time()
            
            # Rate limiting
            time.sleep(1.5)
            
            # Print progress every minute
            if current_time - last_update_time >= 60:
                print(f"Progress: {summoners_processed} summoners processed.")
                last_update_time = current_time

    # Final progress update
    print(f"Final Progress: {summoners_processed} summoners processed. Task complete.")

# Example usage


#source_csv_path = '../DataProcessing/TestData/SILVER_players_korea.csv'
#target_csv_path = '../DataProcessing/TestData/SILVER_GAMES_KOREA.csv'
#fetch_matches_and_save(source_csv_path, target_csv_path)

Progress: 21 summoners processed.
Progress: 41 summoners processed.
Progress: 61 summoners processed.
Progress: 81 summoners processed.
Progress: 101 summoners processed.
Progress: 121 summoners processed.
Progress: 141 summoners processed.
Progress: 161 summoners processed.
Progress: 181 summoners processed.
Progress: 201 summoners processed.
Progress: 221 summoners processed.
Progress: 241 summoners processed.
Progress: 261 summoners processed.
Progress: 281 summoners processed.
Progress: 301 summoners processed.
Progress: 321 summoners processed.
Progress: 341 summoners processed.
Progress: 361 summoners processed.
Progress: 381 summoners processed.
Progress: 401 summoners processed.
Progress: 421 summoners processed.
Progress: 441 summoners processed.
Progress: 461 summoners processed.
Progress: 481 summoners processed.
Progress: 501 summoners processed.
Progress: 521 summoners processed.
Progress: 541 summoners processed.
Progress: 561 summoners processed.
Progress: 581 summoners 

def remove_duplicate_matches(csv_file_path, output_file_path):
    
    Removes duplicate records from a CSV file based on unique MatchID values and saves the cleaned data to a new CSV file.
    
    This function reads a CSV file into a pandas DataFrame, removes duplicate entries based on the 'MatchID' column by      keeping only the first occurrence of each duplicate, and then saves the cleaned DataFrame to a specified output CSV     file. A confirmation message is printed to indicate the successful saving of the cleaned CSV file.
    
    Parameters:
    - csv_file_path (str): The file path to the input CSV file containing the original dataset with potential duplicate               matches.
    - output_file_path (str): The file path where the cleaned dataset without duplicates will be saved.
    
    Returns:
    None
    


In [36]:
def remove_duplicate_matches(csv_file_path, output_file_path):
    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Remove duplicates based on the 'MatchID' column and keep the first occurrence
    cleaned_df = df.drop_duplicates('MatchID', keep='first')
    
    # Save the cleaned DataFrame to a new CSV file
    cleaned_df.to_csv(output_file_path, index=False)
    
    print(f"Cleaned CSV saved to {output_file_path}.")
    
csv_file_path='../DataProcessing/TestData/IRON_GAMES_KOREA.csv'
output_file_path='../DataProcessing/TestData/CLEANED_IRON_GAMES_KOREA.csv'
remove_duplicates_and_save(csv_file_path, output_file_path)

csv_file_path='../DataProcessing/TestData/BRONZE_GAMES_KOREA.csv'
output_file_path='../DataProcessing/TestData/CLEANED_BRONZE_GAMES_KOREA.csv'
remove_duplicates_matches(csv_file_path, output_file_path)

#source_csv_path = "../DataProcessing/TestData/GOLD_players.csv"



Cleaned CSV saved to ../DataProcessing/TestData/CLEANED_IRON_GAMES_KOREA.csv.
Cleaned CSV saved to ../DataProcessing/TestData/CLEANED_BRONZE_GAMES_KOREA.csv.
Cleaned CSV saved to ../DataProcessing/TestData/CLEANED_SILVER_GAMES_KOREA.csv.


def call_api_and_save_response_with_progress(csv_file_path, save_folder_path):
    """
    Calls an API for each match ID listed in a CSV file and saves the response to a JSON file in the specified folder, with progress updates.
    
    This function reads match IDs from a CSV file and makes API calls for each ID to fetch match data. The responses are    saved as JSON files in a specified directory. The function handles rate limiting by ensuring that no more than 30       requests are made per second. It provides progress updates every minute and a final completion message once all matches are processed.
    
    Parameters:
    - continent (str): the target continent of matchids. europe for EUW1_matches for, asia for KR.
    - csv_file_path (str): The file path to the input CSV file containing match IDs.
    - save_folder_path (str): The directory path where the JSON files with the API responses will be saved.
    
    The function implements rate limiting and error handling for HTTP response codes, ensuring compliance with the API's     request constraints. Progress updates are printed to the console, indicating the number of matches processed and the    overall completion percentage.
    
    Returns:
    None

In [11]:
def call_timelineapi_and_save_response_with_progress(continent,csv_file_path, save_folder_path):
    # Define the base URL and updated headers for the API call
    base_url = f"https://{continent}.api.riotgames.com/lol/match/v5/matches/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
        "Accept-Language": "da-DK,da;q=0.9,en-US;q=0.8,en;q=0.7",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": "productionkey"
    }
    
    # Read Match IDs from the CSV file
    with open(csv_file_path, mode='r', newline='', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        match_ids = [row['MatchID'] for row in csv_reader]
    
    total_matches = len(match_ids)
    matches_processed = 0
    start_time = time.time()
    last_update_time = start_time
    request_times = []  # Track the times of the last 30 requests

    # Process each Match ID
    for match_id in match_ids:
        current_time = time.time()
        
        # Check if we have made 30 requests in the last second
        if len(request_times) >= 30:
            # Calculate the time to wait before making the next request
            time_to_wait = 1 - (current_time - request_times[0])
            if time_to_wait > 0:
                time.sleep(time_to_wait)
        
        full_url = base_url + match_id + "/timeline"
        
        # Make the API call
        response = requests.get(full_url, headers=headers)
        
        # Update the request_times list
        request_times.append(time.time())
        if len(request_times) > 30:
            request_times.pop(0)  # Remove the oldest request time
        
        if response.status_code == 200:
            # Save the response body as a JSON file
            file_path = os.path.join(save_folder_path, f"{match_id}.json")
            with open(file_path, 'w', encoding='utf-8') as json_file:
                json.dump(response.json(), json_file, ensure_ascii=False, indent=4)
        else:
            print(f"Failed to fetch data for Match ID {match_id}. Status Code: {response.status_code}")
        
        matches_processed += 1
        
        if current_time - last_update_time >= 60:
            print(f"Processed {matches_processed}/{total_matches} matches. {int((matches_processed/total_matches)*100)}% complete.")
            last_update_time = current_time
    
    # Final update
    print(f"All {total_matches} matches processed. 100% complete.")

sgamespath='E:\\DistanceTesting\\filenames.csv'
jsonfilepaths='E:\\DistanceTesting\\filenamesv2'
continent = 'europe'
#continent='asia'
call_timelineapi_and_save_response_with_progress(continent,sgamespath, jsonfilepaths)

All 40 matches processed. 100% complete.


def fetch_and_save_match_data(continent, csv_file_path, output_folder_path):
    """
    Fetches match data from a Riot Games API for a specified continent and saves the response bodies to individual JSON files in an output folder.
    
    This function iterates through match IDs listed in a provided CSV file, constructs API requests to fetch match data from the Riot Games API for the specified continent, and saves each successful response as a JSON file in the designated output folder. It handles basic rate limiting by pacing the requests and provides periodic progress updates after every 250 requests or upon completing all requests. The function also ensures the output folder exists, creating it if necessary.
    
    Parameters:
    - continent (str): The continent from which to fetch match data (e.g., 'europe', 'asia').
    - csv_file_path (str): The file path to the input CSV file containing match IDs.
    - output_folder_path (str): The directory path where the output JSON files will be saved.
    
    The function includes error handling for unsuccessful API requests and ensures compliance with the API's rate limit by sleeping briefly between requests. Progress updates are printed to console to track the advancement of the data fetching process.
    
    Returns:
    None
    """


In [10]:
#function for calling and saving responsebody from postmatch data api
def fetch_and_save_postmatch_data(continent, csv_file_path, output_folder_path):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": "production key"
    }

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # Count total MatchIDs in the CSV file
    with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        total_ids = sum(1 for row in reader)

    # Reset file pointer to start
    with open(csv_file_path, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        current_position = 0
        for row in reader:
            match_id = row[0]
            url = f"https://{continent}.api.riotgames.com/lol/match/v5/matches/{match_id}"

            response = requests.get(url, headers=headers)
            current_position += 1
            
            if response.status_code == 200:
                output_file_path = os.path.join(output_folder_path, f"{match_id}.json")
                with open(output_file_path, 'w', encoding='utf-8') as outfile:
                    outfile.write(response.text)
            else:
                print(f"Failed to fetch data for MatchID: {match_id}, Status Code: {response.status_code}")

            # Progress update
            if current_position % 250 == 0 or current_position == total_ids:  # Update every 250 requests or on last request
                print(f"Progress: {current_position}/{total_ids}")

            time.sleep(1 / 30)  # Respect the rate limit

    print(f"Finished processing. Total requests: {current_position}.")
    
csv_file_path='E:\\DistanceTesting\\filenames.csv'
continent = 'europe'
#continent='asia'
output_folder_path='E:\\DistanceTesting\\filenamesv2'
fetch_and_save_postmatch_data(continent,csv_file_path, output_folder_path)

Failed to fetch data for MatchID: MatchID, Status Code: 404
Progress: 41/41
Finished processing. Total requests: 41.


In [10]:
def fetch_one_postmatch_and_save_data(continent, matchID, output_folder_path):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"  # Make sure to replace with your actual API token
    }

    # Ensure the output directory exists
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # Construct the request URL
    url = f"https://{continent}.api.riotgames.com/lol/match/v5/matches/{matchID}"
    
    # Make the request
    response = requests.get(url, headers=headers)
    
    # Check the response status
    if response.status_code == 200:
        output_file_path = os.path.join(output_folder_path, f"{matchID}.json")
        with open(output_file_path, 'w', encoding='utf-8') as outfile:
            outfile.write(response.text)
        print(f"Data for MatchID: {matchID} has been saved successfully.")
    else:
        print(f"Failed to fetch data for MatchID: {matchID}, Status Code: {response.status_code}")

    print(f"Finished processing {matchID}")

# Example usage
continent = 'europe'
matchID = 'EUW1_6864052490'
output_folder_path = 'E:\\DistanceTesting\\filenamesv2'
fetch_one_postmatch_and_save_data(continent, matchID, output_folder_path)


Data for MatchID: EUW1_6864052490 has been saved successfully.
Finished processing EUW1_6864052490


In [6]:
def get_one_timeline_from_matchID(continent, matchID, output_folder_path):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
        "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
        "Origin": "https://developer.riotgames.com",
        "X-Riot-Token": "RGAPI-e75b46d3-5e51-4829-b978-68790c3ebf56"  # Make sure to replace with your actual API token
    }

    # Ensure the output directory exists
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # Construct the request URL
    url = f"https://{continent}.api.riotgames.com/lol/match/v5/matches/{matchID}/timeline"
    
    # Make the request
    response = requests.get(url, headers=headers)
    
    # Check the response status
    if response.status_code == 200:
        output_file_path = os.path.join(output_folder_path, f"{matchID}.json")
        with open(output_file_path, 'w', encoding='utf-8') as outfile:
            outfile.write(response.text)
        print(f"Data for MatchID: {matchID} has been saved successfully.")
    else:
        print(f"Failed to fetch data for MatchID: {matchID}, Status Code: {response.status_code}")

    print(f"Finished processing {matchID}")

# Example usage
continent = 'europe'
matchID = 'EUW1_6871911174'
output_folder_path = 'E:\\DistanceTesting\\filenamesv2'
get_one_timeline_from_matchID(continent, matchID, output_folder_path)


Data for MatchID: EUW1_6871911174 has been saved successfully.
Finished processing EUW1_6871911174
